In [27]:
import pandas as pd
import numpy as np 
pd.options.display.max_colwidth = 100

### RESTAURANTES UBER EATS ECUADOR. SCRAPPEADOS: 9 JUNIO 2020 4-5 PM.

### Se carga la lista de restaurantes scrappeados:

In [28]:
restaurantes=pd.read_csv("restaurantes_ecuador09062020pm.csv")

In [29]:
print("Se tienen: ", len(restaurantes), " restaurantes")

Se tienen:  2425  restaurantes


In [30]:
restaurantes.head()

,Unnamed: 0,Nombre,Categoria_Especifica,Calificacion,Reviews,Direccion,Url
0,0,Cafe Ibiza,$$ • Cafetería,NaN,NaN,"Café Ibiza (Downtown ~ Ambato), Tungurahua • Más información",https://www.ubereats.com/ec/ambato/food-delivery/cafe-ibiza/n0vldW1hS12fOXRReaZ6AA
1,1,Buffalos (Paseo Shopping),$$,NaN,NaN,"El Paseo Shopping Ambato, Tungurahua • Más información",https://www.ubereats.com/ec/ambato/food-delivery/buffalos-paseo-shopping/_LUVReTgQlSVVCMtlWrG6A
2,2,El Parrillero,$ • BBQ,NaN,NaN,"Ambato 180202, Ecuador, • Más información",https://www.ubereats.com/ec/ambato/food-delivery/el-parrillero/bNgI8lKvRjmKCRBUIVQFyA
3,3,La Vacona Grill,$$ • BBQ • Opciones para alérgicos,NaN,NaN,"La Vacona Grill, Ambato, Tungurahua 180103 • Más información",https://www.ubereats.com/ec/ambato/food-delivery/la-vacona-grill/ouxlYUN2Seik5AZgC3RqEg
4,4,Buffalos (Mall de los Andes),$$ • Pollo • BBQ,NaN,NaN,"Mall Los Andes, Ambato, Tungurahua 180207 • Más información",https://www.ubereats.com/ec/ambato/food-delivery/buffalos-mall-de-los-andes/TWtrQsbgQpuc5HKmt7SYMA


### Existen resturantes que figuran en varias categorías generales, y tendrán registros duplicados, se eliminan:

In [31]:
restaurantes.drop("Unnamed: 0" ,axis=1, inplace=True) ##Borrar columna innecesaria que proviene de la extracción.
restaurantes.drop_duplicates(keep="first",inplace=True) ## Borrar duplicados

### Se obtienen finalmente 1202 restaurantes diferentes

In [32]:
restaurantes.shape ##Tamaño del dataset

(1202, 6)

### Variables:

In [33]:
restaurantes.columns ##Columnas

Index(['Nombre', 'Categoria_Especifica', 'Calificacion', 'Reviews',
       'Direccion', 'Url'],
      dtype='object')

### * NOMBRE: 
Nombre con el cuál se identifica el restaurante en la página. (String)
### * CATEGORIA_ ESPECIFICA:
 Diferentes categorías en las cuales se puede ubicar el restaurante. Puede ser multivaluado. Es necesario hacer una limpieza, ya que los datos vienen sucios. (string)

### * CALIFICACIÓN: 
Puntuación que reporta la página para el restaurante. (float)

### * REVIEWS: 
Número de reviews que presenta la página para el restaurante. Es necesario hacer limpieza, ya que viene con caracteres sucios. (String)

### * Dirección:
 Ubicación que reporta la página para el restaurante. Es necesario hacer limpieza, ya que viene con datos sucios.

### * URL: 
URL exacta con la cuál se identifica el restaurante en la página.


### Se observa el dtypes:

In [34]:
print("Tipos de datos: ") 
restaurantes.dtypes ##Tipos de las variables

Tipos de datos: 


Nombre                   object
Categoria_Especifica     object
Calificacion            float64
Reviews                  object
Direccion                object
Url                      object
dtype: object

### Buscar valores faltantes: 

In [35]:
print("Datos faltantes por cada variable: ")
restaurantes.isna().sum() ##Mostrar faltantes por variable

Datos faltantes por cada variable: 


Nombre                    0
Categoria_Especifica      6
Calificacion            604
Reviews                 604
Direccion                 0
Url                       0
dtype: int64

### CALIFICACIÓN Y REVIEWS FALTANTES:
 La mayor cantidad de los faltantes en los registros, está en calificación y número de reviews. Esto es porque la mayor parte de los restaurantes cuando están cerrados ocultan su calificación y número de reviews. Adicional a eso, cuando un restaurante es nuevo en la página, tampoco reporta calificación y reviews. Finalmente, dada la situación actual (covid), muchos restaurantes no están operando y salen no disponibles en la plataforma; por ende, ocultan su información de calificación y reviews.

### RESTO DE VARIABLES FALTANTES:
El resto de variables faltantes en los registros, que son muy pocas, es porque el restaurante no tenía la información disponible.

### LIMPIEZA DE DATOS:

#### * DIRECCIÓN:

In [36]:
restaurantes["Direccion"].head()

0    Café Ibiza (Downtown ~ Ambato), Tungurahua • Más información
1          El Paseo Shopping Ambato, Tungurahua • Más información
2                      Ambato 180202, Ecuador,  • Más información
3    La Vacona Grill, Ambato, Tungurahua 180103 • Más información
4     Mall Los Andes, Ambato, Tungurahua 180207 • Más información
Name: Direccion, dtype: object

*Se observa que tienen el string " • Mas información" en todos lo registros. Se procede a eliminar. Así mismo se borran espacios en blanco al comienzo y final.


In [37]:
restaurantes["Direccion"]=restaurantes["Direccion"].apply(lambda x : str(x).replace("•", "").replace("Más información","").strip()) ##Borrar string y espacios no deseados

In [38]:
restaurantes["Direccion"][1] ##Prueba

'El Paseo Shopping Ambato, Tungurahua'

### * REVIEWS:

In [39]:
restaurantes["Reviews"].value_counts() ##Tipos de valores en reviews

(500+)    33
(12)      20
(14)      16
(11)      16
(15)      15
          ..
(389)      1
(230)      1
(137)      1
(177)      1
(126)      1
Name: Reviews, Length: 192, dtype: int64

Se nota que hay paréntesis y signos "+" no deseados, pues el interes está únicamente en el valor entero. Se procede a eliminar el signo "+" y los parentesis. Adicional a eso, se hace la conversión a columna numérica.

In [40]:
restaurantes["Reviews"]=restaurantes["Reviews"].apply(lambda x : str(x).replace("+", "").replace("(","").replace(")","").strip()) ##Borrar string y espacios

In [41]:
restaurantes['Reviews'] = restaurantes.Reviews.astype('float').astype("Int64") ##Convertir a tipo entero
print(restaurantes["Reviews"].dtype)
restaurantes["Reviews"].head()


Int64


0    <NA>
1    <NA>
2    <NA>
3    <NA>
4    <NA>
Name: Reviews, dtype: Int64

### CATEGORÍA ESPECIFICA:

*Puede verse que todas los registros tienen un " \$ " o " \$\$ ".
 No sé sabe con seguridad lo que significa esto, pero parece importante almacenar si tiene uno o dos. Se retirarán dichos signos del registro de categoría, pero se creará otra variable que diga cuantos "$" tenía.

 *Adicional a esto, también se ve la presencia del carater " • "

In [42]:
restaurantes["Categoria_Especifica"].head()

0                        $$ • Cafetería
1                                    $$
2                               $ • BBQ
3    $$ • BBQ • Opciones para alérgicos
4                      $$ • Pollo • BBQ
Name: Categoria_Especifica, dtype: object

In [43]:
restaurantes['Cantidad_$']=restaurantes["Categoria_Especifica"].apply(lambda x: str(x).count("$")) ## Obtener conteo de $
restaurantes["Categoria_Especifica"]=restaurantes["Categoria_Especifica"].apply(lambda x: str(x).replace("$","")) ##Borrar $

Quedó correctamente configurada la nueva variable:

In [44]:
restaurantes["Cantidad_$"].head()

0    2
1    2
2    1
3    2
4    2
Name: Cantidad_$, dtype: int64

Ahora se procede a tratar con el caracter "•" , parece indicar que es el encargado de separar cuando un restaurante pertenece a múltiples categorías. Se evidencia en la siguiente celda:

* Se observan registros como: "• Asiática • Japonesa • Tailandesa • Sushi • Saludable" , donde el carácter "• " separa categorías. Es importante entonces, antes de borrar dicho carácter, lograr atrapar las diferentes categorías que posee el restaurante.

In [45]:
restaurantes["Categoria_Especifica"][44:50] ##Mostrar evidencia

49                          • Argentina
50    Mariscos • Latina • Latina fusión
51                           • Mexicana
52                                  nan
54                     • Pollo • Alitas
56                              • Pollo
Name: Categoria_Especifica, dtype: object

* Haciendo uso del " • ", se puede hacer un split de cada string con base en dicho carácter, para así guardar las categorías como una lista. Adicional a eso, se puede crear otra variable que indique el número de categorías a las que pertenece cada restaurante: 

In [46]:
restaurantes["Categoria_Especifica"]=restaurantes["Categoria_Especifica"].apply(lambda x: str(x).strip().split("•")[1:]) ## Hacer un split con base en el carácter, obtener desde la segunda hasta la última posición (la primera es vacío)
restaurantes["Numero_Categorias"]=restaurantes["Categoria_Especifica"].apply(lambda x: len(x)) ##Obtener el tamaño de cada lista
restaurantes.head()

,Nombre,Categoria_Especifica,Calificacion,Reviews,Direccion,Url,Cantidad_$,Numero_Categorias
0,Cafe Ibiza,[ Cafetería],NaN,<NA>,"Café Ibiza (Downtown ~ Ambato), Tungurahua",https://www.ubereats.com/ec/ambato/food-delivery/cafe-ibiza/n0vldW1hS12fOXRReaZ6AA,2,1
1,Buffalos (Paseo Shopping),[],NaN,<NA>,"El Paseo Shopping Ambato, Tungurahua",https://www.ubereats.com/ec/ambato/food-delivery/buffalos-paseo-shopping/_LUVReTgQlSVVCMtlWrG6A,2,0
2,El Parrillero,[ BBQ],NaN,<NA>,"Ambato 180202, Ecuador,",https://www.ubereats.com/ec/ambato/food-delivery/el-parrillero/bNgI8lKvRjmKCRBUIVQFyA,1,1
3,La Vacona Grill,"[ BBQ , Opciones para alérgicos]",NaN,<NA>,"La Vacona Grill, Ambato, Tungurahua 180103",https://www.ubereats.com/ec/ambato/food-delivery/la-vacona-grill/ouxlYUN2Seik5AZgC3RqEg,2,2
4,Buffalos (Mall de los Andes),"[ Pollo , BBQ]",NaN,<NA>,"Mall Los Andes, Ambato, Tungurahua 180207",https://www.ubereats.com/ec/ambato/food-delivery/buffalos-mall-de-los-andes/TWtrQsbgQpuc5HKmt7SYMA,2,2


* URL:
La URL no necesita ninguna corrección, solo se usará para obtener el nombre de la ciudad donde está el restaurante. Dicho nombre de la ciudad está explicito en la URL. Se almacenará dicha ciudad en una nueva variable ciudad. Para lograr esto se hará un juego con el string de la URL, conocida ya su estructura:

In [47]:
restaurantes["Ciudad"]=restaurantes["Url"].apply(lambda a: a[a.index("/ec/"):a.index("/food-")].split("/")[2].capitalize() )
##Obtener la ciudad, jugando con los indices de la estructura de texto conocida
restaurantes.head(3)


,Nombre,Categoria_Especifica,Calificacion,Reviews,Direccion,Url,Cantidad_$,Numero_Categorias,Ciudad
0,Cafe Ibiza,[ Cafetería],NaN,<NA>,"Café Ibiza (Downtown ~ Ambato), Tungurahua",https://www.ubereats.com/ec/ambato/food-delivery/cafe-ibiza/n0vldW1hS12fOXRReaZ6AA,2,1,Ambato
1,Buffalos (Paseo Shopping),[],NaN,<NA>,"El Paseo Shopping Ambato, Tungurahua",https://www.ubereats.com/ec/ambato/food-delivery/buffalos-paseo-shopping/_LUVReTgQlSVVCMtlWrG6A,2,0,Ambato
2,El Parrillero,[ BBQ],NaN,<NA>,"Ambato 180202, Ecuador,",https://www.ubereats.com/ec/ambato/food-delivery/el-parrillero/bNgI8lKvRjmKCRBUIVQFyA,1,1,Ambato


In [48]:
restaurantes=restaurantes[["Nombre","Ciudad","Categoria_Especifica","Numero_Categorias","Cantidad_$","Calificacion","Reviews","Direccion","Url"]]

In [49]:
restaurantes.reset_index(inplace=True) ##Organizar indices del dataset
restaurantes.drop("index",axis=1,inplace=True) #borrar indice viejo
restaurantes.head(3) ##Muestradefinitiva. 



,Nombre,Ciudad,Categoria_Especifica,Numero_Categorias,Cantidad_$,Calificacion,Reviews,Direccion,Url
0,Cafe Ibiza,Ambato,[ Cafetería],1,2,NaN,<NA>,"Café Ibiza (Downtown ~ Ambato), Tungurahua",https://www.ubereats.com/ec/ambato/food-delivery/cafe-ibiza/n0vldW1hS12fOXRReaZ6AA
1,Buffalos (Paseo Shopping),Ambato,[],0,2,NaN,<NA>,"El Paseo Shopping Ambato, Tungurahua",https://www.ubereats.com/ec/ambato/food-delivery/buffalos-paseo-shopping/_LUVReTgQlSVVCMtlWrG6A
2,El Parrillero,Ambato,[ BBQ],1,1,NaN,<NA>,"Ambato 180202, Ecuador,",https://www.ubereats.com/ec/ambato/food-delivery/el-parrillero/bNgI8lKvRjmKCRBUIVQFyA


In [50]:
restaurantes.to_excel('Ecuador_Limpios.xlsx', encoding='utf-8')